In [1]:
import pandas as pd
import re

In [2]:
tsla = pd.read_csv('TSLA(2).csv')
elon = pd.read_csv('elonmusk.csv')

In [3]:
def separate_date(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()
    
    return date

In [4]:
def separate_time(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()

    return time


In [5]:
tsla['Date'] = tsla['Date'].apply(lambda x : pd.Timestamp(x).date())

In [6]:
elon['Date'] = elon.apply(lambda x: separate_date(x['Timestamp']), axis=1)

In [7]:
elon['Time'] = elon.apply(lambda x: separate_time(x['Timestamp']), axis=1)

In [8]:
elon['weekday'] = elon['Date'].apply(lambda x: x.weekday())

In [9]:
elon['is_weekend'] = elon['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

In [10]:
elon['trading_day'] = 0
elon.loc[elon['Date'].isin(tsla['Date']), ['trading_day']] = 1



In [22]:
tsla[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = tsla[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].pct_change()
    

In [11]:
def getFivePriorTradingDays(date):
    priors = tsla[tsla['Date'] < date]
    return priors['Date'].tail(5).tolist()
    
    

In [106]:
getFivePriorTradingDays(pd.Timestamp('2020-01-01').date())

[datetime.date(2019, 12, 24),
 datetime.date(2019, 12, 26),
 datetime.date(2019, 12, 27),
 datetime.date(2019, 12, 30),
 datetime.date(2019, 12, 31)]

In [12]:
def getValues(dates, col):
    res = []
    for date in dates:
        res.append(tsla[tsla['Date'] == date][col].values[0])
    
    return res

In [13]:
elon['five_prior_trading_days'] = elon['Date'].apply(lambda x: getFivePriorTradingDays(x))

In [14]:
elon['five_prior_close'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Close'), axis=1)

In [15]:
elon['five_prior_open'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Open'), axis=1)


In [17]:
usefulElon = elon[['text','Comments','Likes','Retweets','Date', 'Time','weekday', 'is_weekend', 'five_prior_trading_days','five_prior_open','five_prior_close', 'trading_day']].loc[1:]

In [108]:
usefulElon.tail(25)

,text,Comments,Likes,Retweets,Date,Time,weekday,is_weekend,five_prior_trading_days,five_prior_open,five_prior_close,trading_day,clean_text
12181,Replying to \n@eddyiskongz\nTurn on Starlink o...,507,"3,361",39.8K,2022-03-03,19:54:48,3,0,"[2022-02-24, 2022-02-25, 2022-02-28, 2022-03-0...","[700.390015, 809.22998, 815.01001, 869.679993,...","[800.77002, 809.869995, 870.429993, 864.369995...",1,replying to @eddyiskongz turn on starlink onl...
12182,Replying to \n@WholeMarsBlog\n @Teslarati\n an...,195,262,"4,616",2022-03-02,02:59:27,2,0,"[2022-02-23, 2022-02-24, 2022-02-25, 2022-02-2...","[830.429993, 700.390015, 809.22998, 815.01001,...","[764.039978, 800.77002, 809.869995, 870.429993...",1,replying to @wholemarsblog @teslarati and ...
12183,Replying to \n@genesimmons\n and \n@JoeBiden\n...,"1,398","4,040",53K,2022-03-03,01:44:12,3,0,"[2022-02-24, 2022-02-25, 2022-02-28, 2022-03-0...","[700.390015, 809.22998, 815.01001, 869.679993,...","[800.77002, 809.869995, 870.429993, 864.369995...",1,replying to @genesimmons and @joebiden actu...
12184,Replying to \n@shortword\n and \n@eddyiskongz\...,188,315,"8,715",2022-03-03,21:17:24,3,0,"[2022-02-24, 2022-02-25, 2022-02-28, 2022-03-0...","[700.390015, 809.22998, 815.01001, 869.679993,...","[800.77002, 809.869995, 870.429993, 864.369995...",1,replying to @shortword and @eddyiskongz yes...
12185,Replying to \n@WholeMarsBlog\nWe could’ve made...,"1,783","1,576",22.2K,2022-03-02,11:19:18,2,0,"[2022-02-23, 2022-02-24, 2022-02-25, 2022-02-2...","[830.429993, 700.390015, 809.22998, 815.01001,...","[764.039978, 800.77002, 809.869995, 870.429993...",1,replying to @wholemarsblog we could’ve made a...
12186,Replying to \n@RealLifeLore22\nPretty good sum...,596,602,"8,507",2022-03-02,08:08:22,2,0,"[2022-02-23, 2022-02-24, 2022-02-25, 2022-02-2...","[830.429993, 700.390015, 809.22998, 815.01001,...","[764.039978, 800.77002, 809.869995, 870.429993...",1,replying to @reallifelore22 pretty good summa...
12187,Replying to \n@ICannot_Enough\n and \n@Model3O...,632,737,"8,131",2022-03-02,00:21:52,2,0,"[2022-02-23, 2022-02-24, 2022-02-25, 2022-02-2...","[830.429993, 700.390015, 809.22998, 815.01001,...","[764.039978, 800.77002, 809.869995, 870.429993...",1,replying to @icannot_enough and @model3owne...
12188,Replying to \n@ICannot_Enough\n @genesimmons\n...,385,895,"6,858",2022-03-03,05:04:31,3,0,"[2022-02-24, 2022-02-25, 2022-02-28, 2022-03-0...","[700.390015, 809.22998, 815.01001, 869.679993,...","[800.77002, 809.869995, 870.429993, 864.369995...",1,replying to @icannot_enough @genesimmons an...
12189,Replying to \n@FedorovMykhailo\n @Brian_J_Berg...,352,473,"7,801",2022-03-02,20:07:25,2,0,"[2022-02-23, 2022-02-24, 2022-02-25, 2022-02-2...","[830.429993, 700.390015, 809.22998, 815.01001,...","[764.039978, 800.77002, 809.869995, 870.429993...",1,replying to @fedorovmykhailo @brian_j_berger...
12190,Replying to \n@elonmusk\n and \n@MSNBC\nSaints...,202,260,"3,852",2022-03-02,03:38:31,2,0,"[2022-02-23, 2022-02-24, 2022-02-25, 2022-02-2...","[830.429993, 700.390015, 809.22998, 815.01001,...","[764.039978, 800.77002, 809.869995, 870.429993...",1,replying to @elonmusk and @msnbc saints row...


In [19]:
def clean_text(x):
    temp = x.lower()
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    temp = re.sub(r"\n", " ", temp)
    temp = temp.rsplit(' ', 3)[0]
    return temp
    
    
    

In [20]:
usefulElon['clean_text'] = usefulElon['text'].apply(lambda x: clean_text(x))

In [81]:
def get_trading_info(row):
    if row['trading_day'] == 1:
        temp = tsla[tsla['Date'] == row['Date']]
    else:
        next_trading_day = tsla[tsla['Date'] > row['Date']].iloc[0]['Date']
        temp = tsla[tsla['Date'] == next_trading_day]
    
    return pd.Series(temp.values[0][1:])

In [82]:
get_trading_info(usefulElon.iloc[0])

0    0.002154
1    0.032189
2   -0.007449
3   -0.004276
4   -0.004276
5    0.354991
dtype: object

In [101]:
labels = usefulElon.copy()
labels[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = labels.apply(
    lambda x: get_trading_info(x), axis=1)


In [102]:
labels = labels[['Date','Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

In [103]:
labels

,Date,Open,High,Low,Close,Adj Close,Volume
1,2011-12-01,0.002154,0.032189,-0.007449,-0.004276,-0.004276,0.354991
2,2011-12-01,0.002154,0.032189,-0.007449,-0.004276,-0.004276,0.354991
3,2011-12-03,0.021322,0.038884,0.031790,0.033634,0.033634,0.445067
4,2011-12-03,0.021322,0.038884,0.031790,0.033634,0.033634,0.445067
5,2011-12-04,0.021322,0.038884,0.031790,0.033634,0.033634,0.445067
...,...,...,...,...,...,...,...
12201,2022-03-03,0.007614,-0.000045,-0.013823,-0.046142,-0.046142,-0.174426
12202,2022-03-03,0.007614,-0.000045,-0.013823,-0.046142,-0.046142,-0.174426
12203,2022-03-02,0.002817,-0.003821,-0.011139,0.017955,0.017955,-0.001653
12204,2022-03-04,-0.033763,-0.034734,-0.008936,-0.001191,-0.001191,0.087239


In [104]:
labels.to_csv('lables.csv')
usefulElon.to_csv('features.csv')